In [15]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
path = "D:\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.clien.net/service/board/cm_havehome")

In [17]:
def crawl_title_and_date(dict_title_and_date):
    
    # 게시글 제목과 날짜를 크롤링한다
    try:
        # 제목 크롤링
        title = driver.find_element_by_css_selector('#div_content > div.post_title.symph_row > h3 > span:nth-child(2)').text
        dict_title_and_date['title'].append(title)
        
        date = driver.find_element_by_css_selector('#div_content > div.post_view > div.post_author > span:nth-child(1)').text[:10]
        dict_title_and_date['date'].append(date)
        
    # 크롤링이 안되면 뒤로 돌아간다.
    except:
        driver.back()
        pass
    
    return dict_title_and_date


In [18]:
def crawl_text(text_ls, datgle_ls):
    # 게시글에 들어온 다음, 크롤링을 진행한다
    try:
        # 게시글 크롤링
        txt = driver.find_element_by_css_selector('#div_content > div.post_view > div.post_content').text.replace('\n', ' ')
        text_ls.append(txt)

        temp_datgle_list = [x.text.replace('\n', '') for x in driver.find_elements_by_class_name('comment_view') if not x.text == '']
        
        if temp_datgle_list:
            datgle_ls += temp_datgle_list

    # 크롤링이 안되면 뒤로 돌아간다.
    except:
        driver.back()
        pass
    
    
    return text_ls, datgle_ls
    


In [43]:
def crawl_one_page(text_ls, datgle_ls, dict_title_and_date):
    import time
    time.sleep(1)
    
    # 개별 게시글에 접근
    for i in range(8,40):
        try:
            # 게시글에 접근
            driver.find_element_by_css_selector('#div_content > div:nth-child(%s) > div.list_title > a.list_subject > span:nth-child(2)'%(i)).click()
            
            time.sleep(0.5)
            
            # 게시글과 댓글 크롤링
            text_ls, datgle_ls = crawl_text(text_ls, datgle_ls)
            dict_title_and_date = crawl_title_and_date(dict_title_and_date)
            
            if not driver.find_element_by_css_selector('#div_content > div.board_head > div.board_name > h2 > a') == '내집마련당':
                driver.back()
            
        except : 
            pass
        
    return text_ls, datgle_ls, dict_title_and_date

In [41]:
def crawl_multiple_pages(start_page, end_page):
    import time 
    
    text_ls = []
    datgle_ls = []
    dict_title_and_date = {'title' : [],
                           'date' : []}
    
    try:   driver.switch_to_frame('cafe_main')
    except:   pass

    for k in range(start_page+1, end_page+1):
        
        # 게시글과 댓글 크롤링
        text_ls, datgle_ls, dict_title_and_date_title = crawl_one_page(text_ls, datgle_ls, dict_title_and_date)
        
        if not k%10 == 1:
            
            # 페이지를 이동한다
            try:
                driver.find_element_by_css_selector('#pagingActiveId_%s'%(k)).click()
                time.sleep(5)
            #안되면 그만한다
            except:
                return text_ls, datgle_ls, dict_title_and_date
            
        elif k%10 == 1:
            
            #다음 장으로 넘어간다
            try:
                driver.find_element_by_css_selector('#div_content > div.list_button_area > div.board-pagination > div > a.board-nav-next').click()
                time.sleep(5)
            except:
                return text_ls, datgle_ls, dict_title_and_date
    
    return text_ls, datgle_ls, dict_title_and_date

In [42]:
text_ls, datgle_ls, dict_title_and_date = crawl_multiple_pages(151, 300)

total_ls = text_ls + datgle_ls
total_df = pd.DataFrame(total_ls,
                        columns = ['A'])

total_df.to_csv('Clien_myhome_text_and_datgle_%s.csv'%(len(total_df)),index = False)

C:\Users\Dong\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: use driver.switch_to.frame instead
  if __name__ == '__main__':


In [44]:
df_title_and_date = pd.DataFrame(dict_title_and_date)

try:
    df_title_and_date['text'] = text_ls
except:
    pass

df_title_and_date.to_csv('Clien_myhome_title_and_date_%s.csv'%(len(df_title_and_date)),index = False)

In [45]:
total_df.head()

,A
0,안녕하세요. 7월중에 이사를 가야하는데 들어오실분이 8월에 올수있다는 애매한 상황...
1,안녕하세요? 부동산 투자에 대해서 감을 잡으려고 질문 좀 드리겠습니다. 4층 또...
2,최근에 청약 당첨되어 계약 완료한 상태입니다. 주택청약을 해지해야 하는 것으로 알고...
3,부동산114에 회원님이 문의한 내용을 바탕으로 만든 자료입니다. 참고하시기 바랍니...
4,"내년 4월말 전세 만기가 됩니다. 저는 다른 아파트 신규분양권을 매입했고, 입주기간..."


In [46]:
len(total_df)

11176